In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

import pickle
import os
import torch

import pandas as pd

import plotly.io as pio
pio.renderers.default = 'notebook+vscode'
from sklearn.cluster import KMeans

#import sys
#sys.path.insert(0, '/Users/danielverschueren/Documents/Sonalis/code/registration/registration/')
#sys.path.insert(0, '/Users/danielverschueren/Documents/Sonalis/code/registration/')

from point_cloud_torch import affinePC_torch, apply_affine_torch, apply_affine_inverse_torch
from point_cloud import plot_PCs, affinePC

# the fit function in affinePC calls on minimize_parallel from optimparallel
# make sure that package is installed


def pgy2npy(pgy_file):
    """
    pgy file to npy
    """
    with open(pgy_file) as f:
        lines = f.readlines()
    lines = lines[1:] # remove header
    np_data = np.zeros((len(lines), 3), np.float64)
    for i, line in enumerate(lines):
        l = []
        for t in line.split():
            try:
                l.append(float(t))
            except ValueError:
                pass
        np_data[i] = np.asarray(l[1:])
    return np_data

plot_using = 'plotly_light'

In [ ]:
# Some useful color arrays
colour_white = np.array([[1, 1, 1]], dtype=np.float64).swapaxes(1, 0)
colour_red = np.array([[1, 0, 0]], dtype=np.float64).swapaxes(1, 0)
colour_black = np.array([[0, 0, 0]], dtype=np.float64).swapaxes(1, 0)
colour_blue = np.array([[0, 0, 1]], dtype=np.float64).swapaxes(1, 0)
colour_green = np.array([[0, 1, 0]], dtype=np.float64).swapaxes(1, 0)

## load cleaned point cloud

In [ ]:
refined_points = np.load('coords/PC-jul24.npy')
# plot and verify
pcd_refined = o3d.utility.Vector3dVector(refined_points)
pcd_refined = o3d.geometry.PointCloud(pcd_refined)
o3d.visualization.draw_geometries([pcd_refined])

## first initial clustering

In [ ]:
# perform k-means clustering: this does not work super well, but can be optimised
# for example by passing rough centers of transducers, changing tolerances, n_init,
# I haven't played with it too much yet.
#
# this shows in open3D much more clearly than with plotly
n_clusters = 1700
kmeans = KMeans(n_clusters=n_clusters, 
                verbose=1, 
                init='k-means++',
                
                n_init='auto', 
                algorithm='elkan',
                random_state=567898).fit(refined_points)

In [ ]:
labels = kmeans.labels_
print(f"point cloud has {labels.max() + 1} clusters")
colors = plt.get_cmap("tab20")(labels /(labels.max() + 1))

# update open3D objects
pcd_refined.colors = o3d.utility.Vector3dVector(colors[:, :3])

# separate clusters
cluster_points = []
for i in range(n_clusters):
    cluster_points.append(refined_points[labels == i][::50])

# visualise
if plot_using == 'plotly':
    pass
    #fig = plot_PCs(cluster_points)
    #fig.show()
elif plot_using == 'open3D':
    o3d.visualization.draw_geometries([pcd_refined])
else:
    pass
    #fig = plot_PCs(cluster_points[::10])
    #fig.write_html('vis_clusters-1.html')

In [ ]:
# plot cluster centers
c1s = kmeans.cluster_centers_
fig = plot_PCs([c1s])
fig.show()

## Register nominal geometry to cluster centers

In [ ]:
# open nominal geometry
nominal_geometry_points = pgy2npy('coords/omega2-PointReceivers_in_mm.pgy')
nominal_geometry_points.T[[1,2]] = nominal_geometry_points.T[[2,1]] # swap axes
nominal_geometry_points.T[[1,0]] = nominal_geometry_points.T[[0,1]] # swap axes
print(len(nominal_geometry_points))

nominal_geometry = o3d.utility.Vector3dVector(nominal_geometry_points)
pcd_nominal_geometry = o3d.geometry.PointCloud(nominal_geometry)

if plot_using == 'plotly':
    fig = plot_PCs([c1s, nominal_geometry_points])
    fig.show()
elif plot_using == 'open3D':
    o3d.visualization.draw_geometries([pcd_refined, pcd_nominal_geometry])
else:
    fig = plot_PCs([c1s, nominal_geometry_points])
    fig.show()

In [ ]:
# align two point clouds
# center both geometries
center_c1s = c1s.mean(axis=0)
center_nominal = nominal_geometry_points.mean(axis=0)

clusters_centered = c1s - center_c1s
nominal_centered = nominal_geometry_points - center_nominal

# affine
start = torch.Tensor([0,0,2000,0,0,0]) # it will be important to have a rough starting point for 
                      # each! the original geomtery will be very helpful 1600

print('start_fitting...')
N_PC = affinePC_torch(torch.Tensor(clusters_centered), torch.Tensor(nominal_centered))
dphi = 1800
N_PC.fit(start=start, 
         bounds=[(0-dphi,0+dphi),(0-dphi,0+dphi),(900-dphi, 900+dphi)]+[(None,None)]*3, 
         method='naive',
         max_oper=1000,
         lr=0.1)
print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{N_PC.params}")
reg_nominal = N_PC.apply_aff()
params = N_PC.params

# construct open3D objects
reg_nom = o3d.utility.Vector3dVector(reg_nominal)
pcd_reg_nom = o3d.geometry.PointCloud(reg_nom)
pcd_reg_nom = pcd_reg_nom.paint_uniform_color(colour_green)
clusters_ = o3d.utility.Vector3dVector(clusters_centered)
pcd_cluster_c = o3d.geometry.PointCloud(clusters_)
pcd_cluster_c = pcd_cluster_c.paint_uniform_color(colour_blue)

o3d.visualization.draw_geometries([pcd_reg_nom, pcd_cluster_c])


## Visualise clusters and centers to identify missing transducers
(note plotly plot numbers start at 1, numpy at 0)

In [ ]:
# construct open3D objects
cluster_centers_vis = [clusters_centered]
for c in reg_nominal:
    cluster_centers_vis.append(c.reshape(1,-1))
fig = plot_PCs(cluster_centers_vis)
fig.write_html('vis_centers.html')

In [ ]:
clust_list = [point.reshape(1,-1) + center_c1s for point in reg_nominal]
clust_list.append(refined_points[::10])
fig = plot_PCs(clust_list)
fig.write_html('vis_clust.html')

In [ ]:
# remove broke transducers
remove = [839, 841, 1064, 1145, 1171, 1326, 1327, 1354, 1379] #[1156, 1524, 1525, 252, 1055]
reg_nominal_clean = []
clust_list_clean = []
reg_removed = []
for i, clust in enumerate(reg_nominal):

    if i in remove:
        reg_removed.append(clust)
        continue

    reg_nominal_clean.append(clust)

# add two more points for random bits of points
#reg_nominal_clean.append((torch.Tensor([-77.2590, 164.5, -159.7]) - center_c1s).float())
#reg_nominal_clean.append((torch.Tensor([31.5, -174, -162]) - center_c1s).float())

print(reg_nominal_clean[-3])
print(reg_nominal_clean[-1])

In [ ]:
# refit with removed pieces
start = torch.Tensor([0,0,0,0,0,0]) # it will be important to have a rough starting point for 
                      # each! the original geomtery will be very helpful [1800,1800,1400,0,0,0]

print('start_fitting...')
N_PC = affinePC_torch(torch.Tensor(clusters_centered), torch.vstack(reg_nominal_clean))
dphi = 1800
N_PC.fit(start=start, 
         bounds=[(0-dphi,0+dphi),(0-dphi, 0+dphi),(0-dphi, 0+dphi)]+[(None,None)]*3, 
         method='naive',
         max_oper=1000,
         lr=0.001)
print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{N_PC.params}")
reg_nominal_clean = N_PC.apply_aff()
params = N_PC.params

# cvisualize fits
reg_nom = o3d.utility.Vector3dVector(reg_nominal_clean)
pcd_reg_nom = o3d.geometry.PointCloud(reg_nom)
pcd_reg_nom = pcd_reg_nom.paint_uniform_color(colour_green)
clusters_ = o3d.utility.Vector3dVector(clusters_centered)
pcd_cluster_c = o3d.geometry.PointCloud(clusters_)
pcd_cluster_c = pcd_cluster_c.paint_uniform_color(colour_blue)

o3d.visualization.draw_geometries([pcd_reg_nom, pcd_cluster_c])

## Clustering round 2: get clusters

In [ ]:
#rerun 1705 minus 9 plus 2?
n_clusters = 1696
reg_nominal_clean_x = reg_nominal_clean.numpy()
#np.random.shuffle(reg_nominal_clean_x)
init_c = reg_nominal_clean_x + center_c1s

# add random fluffs
craps = np.array([[53.0, -172.0, -164.0],
                  [72.0, 164.0, -162.0]])
#init_c = np.vstack([init_c, craps])

kmeans_rerun = KMeans(n_clusters=n_clusters, 
                      verbose=1, 
                      init=init_c,
                      n_init=1, 
                      algorithm='elkan').fit(refined_points)

In [ ]:
labels_rerun = kmeans_rerun.labels_
print(f"point cloud has {labels_rerun.max() + 1} clusters")
colors = plt.get_cmap("tab20")(labels_rerun /(labels_rerun.max() + 1))

# update open3D objects
pcd_refined.colors = o3d.utility.Vector3dVector(colors[:, :3])

# separate clusters
cluster_rerun_points = []
for i in range(labels_rerun.max()+1):
    x = refined_points[labels_rerun == i]
    cluster_rerun_points.append(x[::1])

o3d.visualization.draw_geometries([pcd_refined])

In [ ]:
# write only with reduced set op points!
#fig=plot_PCs(cluster_rerun_points)
#fig.write_html('clusters.html')

In [ ]:
# save clusters
clusters_rerun_points_labelled = cluster_rerun_points[0]
lab = np.ones(len(clusters_rerun_points_labelled))*0
clusters_rerun_points_labelled = np.c_[clusters_rerun_points_labelled, lab]

for i in range(1,len(cluster_rerun_points)):
    temp_np = cluster_rerun_points[i]
    lab = np.ones(len(temp_np))*i
    temp_np = np.c_[temp_np, lab]
    clusters_rerun_points_labelled = np.vstack((clusters_rerun_points_labelled, temp_np))

print(clusters_rerun_points_labelled[-5:])
clusters_rerun_points_labelled.tofile('ExtractedTransducerClustersPCs-jul24.npy')

bucket = {"points" : refined_points,
          "labels"  : labels_rerun}
with open(f"ClusteredPCs-jul24.pk", 'wb') as handle:
    pickle.dump(bucket, handle)